# KPI Visualizer

Creates overview KPI .xls by data manipulation of available monthly data points.

## Data processing

### Import and init modules:

In [1]:
import os
import openpyxl
from openpyxl.styles import Font, Color
from openpyxl.utils.dataframe import dataframe_to_rows

import datascience as ds
from datascience.predicates import are
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

### List available KPI data points

In [2]:
files = os.listdir()
monthly_kpi = [f for f in files if f.startswith('WKPL_KPI_')]
kpi_count = len(monthly_kpi)

### Create different views

#### Create excel file to store data

In [3]:
excel = openpyxl.Workbook()
source_sheet = excel.active
source_sheet.title = 'Source Data'
excel.create_sheet(title='Contact Count')
excel.create_sheet(title='Email Activity (M)')
excel.create_sheet(title='Form Activity (M)')
excel.create_sheet(title='Blacklist Count (M)')
excel.create_sheet(title='Not Opened 10+ (Y)')
excel.create_sheet(title='Cookie Linked (A)')
excel.sheetnames

['Source Data',
 'Contact Count',
 'Email Activity (M)',
 'Form Activity (M)',
 'Blacklist Count (M)',
 'Not Opened 10+ (Y)',
 'Cookie Linked (A)']

#### Storing source data

In [4]:
for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    kpi_csv.relabel('Segment', kpi_date)
    # Save all data points to source sheet
    df = kpi_csv.to_df()
    for row in dataframe_to_rows(df, index=False, header=True):
        source_sheet.append(row)
    # Add spacer
    source_sheet.append([])

In [5]:
# Base array with segments
segment_column = ds.make_array(
        'ALL + WMR', 'ALL - WMR',
        'LEG + WMR', 'LEG - WMR',
        'PUB + WMR', 'PUB - WMR',
        'FIR + WMR', 'FIR - WMR',
        'OTH + WMR', 'OTH - WMR')

#### Contact Count View

In [6]:
contact_sheet = excel['Contact Count']
contact_sheet.freeze_panes = 'B2'

contact_view = ds.Table().with_column('Contact Count', segment_column)

for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][3:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    # Save all data points
    contact_view.append_column(f'Contacts {kpi_date}', kpi_csv.column('Contacts'))
    contact_view.append_column(f'NSL {kpi_date}', kpi_csv.column('NSL'))
    contact_view.append_column(f'NSL Prof {kpi_date}', kpi_csv.column('NSL Prof'))
    contact_view.append_column(f'Alert {kpi_date}', kpi_csv.column('Alert'))

# Sorts columns by type
column_selector = [0]
for i in range(4):
    column_selector.extend(range(1 + i, 4 * kpi_count + 1, 4))
contact_view = contact_view.select(column_selector)

# Saves data to excel file
df = contact_view.to_df()
for row in dataframe_to_rows(df, index=False, header=True):
    contact_sheet.append(row)

#### Email Activity View

In [7]:
email_sheet = excel['Email Activity (M)']
email_sheet.freeze_panes = 'B2'

email_view = ds.Table().with_column('Email Activity', segment_column)

for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][3:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    # Save all data points
    email_view.append_column(f'Sent {kpi_date}', kpi_csv.column('Sent (M)'))
    email_view.append_column(f'Open {kpi_date}', kpi_csv.column('Open (M)'))
    email_view.append_column(f'Click {kpi_date}', kpi_csv.column('Click (M)'))

# Sorts columns by type
column_selector = [0]
for i in range(3):
    column_selector.extend(range(1 + i, 3 * kpi_count + 1, 3))
email_view = email_view.select(column_selector)

# Saves data to excel file
df = email_view.to_df()
for row in dataframe_to_rows(df, index=False, header=True):
    email_sheet.append(row)

#### Form Activity View

In [8]:
form_sheet = excel['Form Activity (M)']
form_sheet.freeze_panes = 'B2'

form_view = ds.Table().with_column('Form Activity', segment_column)

for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][3:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    # Save all data points
    form_view.append_column(kpi_date, kpi_csv.column('Form (M)'))

# Saves data to excel file
df = form_view.to_df()
for row in dataframe_to_rows(df, index=False, header=True):
    form_sheet.append(row)

#### Blacklist View

In [9]:
blacklist_sheet = excel['Blacklist Count (M)']
blacklist_sheet.freeze_panes = 'B2'

blacklist_view = ds.Table().with_column('Blacklist Count', segment_column)

for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][3:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    # Save all data points
    blacklist_view.append_column(kpi_date, kpi_csv.column('Unsub (M)'))

# Saves data to excel file
df = blacklist_view.to_df()
for row in dataframe_to_rows(df, index=False, header=True):
    blacklist_sheet.append(row)

#### Not Opened View

In [10]:
not_opened_sheet = excel['Not Opened 10+ (Y)']
not_opened_sheet.freeze_panes = 'B2'

not_opened_view = ds.Table().with_column('Not Opened 10+', segment_column)

for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][3:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    # Save all data points
    not_opened_view.append_column(kpi_date, kpi_csv.column('Not Opened 10+ (Y)'))

# Saves data to excel file
df = not_opened_view.to_df()
for row in dataframe_to_rows(df, index=False, header=True):
    not_opened_sheet.append(row)

#### Cookie Link Count View

In [11]:
cookie_sheet = excel['Cookie Linked (A)']
cookie_sheet.freeze_panes = 'B2'

cookie_view = ds.Table().with_column('Cookie Linked', segment_column)

for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][3:-4]
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    # Save all data points
    cookie_view.append_column(kpi_date, kpi_csv.column('Cookie Linked (A)'))

# Saves data to excel file
df = cookie_view.to_df()
for row in dataframe_to_rows(df, index=False, header=True):
    cookie_sheet.append(row)

## Saving data to excel

In [12]:
excel.save('WKPL_KPI.xlsx')